# Model things

In [1]:
from src.evaluation.metrics import competition_metric
from src.preprocessing.data_formatting import get_preprocessed_data
import pandas as pd
import numpy as np
import re

df_train = get_preprocessed_data("../input", skip_preprocess=False, filename="train")
df_test = pd.read_csv("../input/raw/testanswer.csv")

del df_train['Index']
df_all = pd.concat([df_train, df_test], ignore_index=True, sort=False)

df_all

,Artist,Track,Album,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,...,AnC,AnD,BnC,BnD,CnD,AnBnC,AnBnD,AnCnD,BnCnD,AnBnCnD
0,tajima hal,Grape Choice,Grape Choice,0.704,0.5870,-9.815,0.3120,0.341000,0.004510,0.1410,...,0,0,0,0,0,0,0,0,0,0
1,CHROMANCE,Wrap Me In Plastic - Marcus Layton Radio Edit,Wrap Me In Plastic (Marcus Layton Radio Edit),0.917,0.6120,-2.183,0.0854,0.238000,0.000023,0.0585,...,0,0,0,0,0,0,0,0,0,0
2,Stevie Wonder,Sir Duke,Songs In The Key Of Life,0.588,0.5680,-9.766,0.0672,0.160000,0.000000,0.0792,...,0,0,0,0,0,0,0,0,0,0
3,MishCatt,Midnight Sun - Acoustic Version,Midnight Sun (Acoustic Version),0.590,0.0991,-15.925,0.0490,0.915000,0.000005,0.1090,...,0,0,0,0,0,0,0,0,0,0
4,Gloria Estefan,Sólo Por Tu Amor,Alma Caribeña,0.561,0.4770,-8.123,0.0355,0.295000,0.000000,0.0778,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8537,The 1975,Lostmyhead,"I like it when you sleep, for you are so beaut...",0.341,0.6710,-7.481,0.0391,0.241000,0.863000,0.1120,...,0,0,0,0,0,0,0,0,0,0
8538,Klô Pelgag,Insomnie,Brol La Suite,0.496,0.3390,-10.512,0.0426,0.650000,0.000049,0.0945,...,0,0,0,0,0,0,0,0,0,0
8539,Barenaked Ladies,Smile - Acoustic,Grinning Streak (Deluxe Version),0.637,0.5050,-7.332,0.0406,0.268000,0.000007,0.1100,...,0,0,0,0,0,0,0,0,0,0
8540,Skillet,Awake and Alive,Awake,0.434,0.9390,-2.983,0.0593,0.042500,0.000000,0.4910,...,0,0,0,0,0,0,0,0,0,0


In [56]:
from src.evaluation.metrics import competition_metric

feature_cols = ['danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'length']
target_cols = ['A', 'B', 'C', 'D']

# X = df[feature_cols].to_numpy()
# y = df[target_cols].to_numpy()
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])


def get_features(df):

    time_sig_X = get_one_hot(df['time_signature'], 6)

    X = np.column_stack([df[feature_cols].to_numpy(), time_sig_X])
    y = df[target_cols].to_numpy()
    return X, y


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from src.preprocessing.data_formatting import fill_remaining_cols
import copy
import xgboost as xgb
import lightgbm as lgb

scaler = StandardScaler()
X_train, y_train = get_features(df_train)
X_test, y_test = get_features(df_test)

param3 = {'objective': 'binary',
         'boosting': 'gbdt',
         'random_state': 42,
         'metric': 'auc',
         'num_threads': -1,
         'num_leaves': 20,
         'max_depth': 8,
         'reg_alpha': 0,
         'reg_lambda': 0,
         'learning_rate' : 0.2,
          'verbosity': -1,
    }
param = {
    'objective': 'binary',
         'boosting': 'gbdt',
         'random_state': 42,
         'metric': 'auc',
         'num_threads': -1,
         'num_leaves': 20,
         'max_depth': 11,
         'reg_alpha': 0,
         'reg_lambda': 0,
         'learning_rate' : 0.1,
          'verbosity': -1,
    }

logRegs = [LogisticRegression(), LogisticRegression(), LogisticRegression(), LogisticRegression()]

df_preds = pd.read_csv("../input/raw/testanswer.csv") #pd.DataFrame(columns=["A", "B", "C", "D", 'AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD', 'AnBnC', 'AnBnD', 'AnCnD', 'BnCnD', 'AnBnCnD'])
df_true = df_preds.copy() # pd.DataFrame(columns=["A", "B", "C", "D", 'AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD', 'AnBnC', 'AnBnD', 'AnCnD', 'BnCnD', 'AnBnCnD'])

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

models = []

for i in range(4):   
    X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train, y_train[:, i])
    valid_data = lgb.Dataset(X_val2, label=y_val2)
    train_data = lgb.Dataset(X_train2, label = y_train2)
    train_data = lgb.Dataset(X_train, label=y_train[:, i])

    mod = lgb.train(param, train_data, 150)
    models.append(mod)
    preds = mod.predict(X_test)
    df_preds[target_cols[i]] = np.array(preds > 0.42).astype(int) #, dtype=np.int)
    df_true[target_cols[i]] = y_test[:, i]
    
fill_remaining_cols(df_preds)
fill_remaining_cols(df_true)
competition_metric(df_preds, df_true)

/home/mas322/anaconda3/envs/cudf-torch/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.5911325342084761

In [ ]:
df_preds.to_csv

In [24]:
competition_metric(df_preds, df_true)

0.5850907506600131

In [40]:
df_true['AnD'].mean()

0.006436512580456407

In [26]:
df_preds['C'] = preds
df_preds

,A,B,C,D,AnB,AnC,AnD,BnC,BnD,CnD,AnBnC,AnBnD,AnCnD,BnCnD,AnBnCnD
0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1705,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1706,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1707,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
